In [ ]:
import os
import pandas as pd
import numpy as np

TEST_PATH = 'C:/Users/js/Desktop/AISPRAK/사회문제 해결형/data/TEST_INPUT/'

TEST_AWS_PATH = 'C:/Users/js/Desktop/AISPRAK/사회문제 해결형/data/TEST_AWS/'

TEST_MERGE = 'C:/Users/js/Desktop/AISPRAK/사회문제 해결형/data/TEST_MERGE/'

In [ ]:
one2one_col = {'정림동' : '오월드', '문창동' : '오월드',  '읍내동' : '장동',  '노은동' : '계룡',  '아름동' : '세종고운',
                '신흥동' : '세종연서',  '공주' : '공주',  '논산' : '논산',  '대천2동' : '대천항',  '홍성읍' : '홍북',
                '예산군' : '예산',  '모종동' : '아산',  '신방동' : '성거',  '성성동' : '성거',  '동문동' : '태안',
                '이원면' : '안도',  '독곶리' : '대산'}

In [ ]:
for train in os.listdir(TEST_PATH):

    df = pd.read_csv(TEST_PATH+train)
    df.rename(columns={'연도': 'year', '일시' : 'date', "측정소": 'place'},inplace=True)
    df['year'] += 2001
    df['date'] = df['year'].astype('str') + "-" + df['date'].astype('str')
    df['year'] = df['year'].astype('object')
    
    merge_df = pd.read_csv(TEST_AWS_PATH+one2one_col[train.split('.')[0]]+'.csv')
    merge_df.rename(columns={'연도': 'year', '일시' : 'date', "지점": 'place','기온(°C)':'temp','풍향(deg)':'wind_direction','풍속(m/s)':'wind_speed','강수량(mm)':'rain','습도(%)':'humid'},inplace=True)
    merge_df['year'] += 2001
    merge_df['date'] = merge_df['year'].astype('str') + "-" + merge_df['date'].astype('str')
    merge_df['year'] = merge_df['year'].astype('object')

    merged_df = pd.merge(df,merge_df,how='inner',on='date')

    merged_df.to_csv(TEST_MERGE+train,index=False)

In [ ]:
final_merged_df = pd.DataFrame()

for lst in os.listdir(TEST_MERGE):
    df = pd.read_csv(TEST_MERGE+lst)
    df.drop(['year_x','place_y','year_y'],inplace=True,axis=1)
    df.rename(columns={'place_x':'place'},inplace=True)

    final_merged_df = pd.concat([final_merged_df,df])

final_merged_df['date'] = pd.to_datetime(final_merged_df['date'], format='%Y-%m-%d %H:%M')
final_merged_df['month'] = final_merged_df['date'].dt.month
# final_merged_df['day'] = final_merged_df['date'].dt.day
# final_merged_df['hour'] = final_merged_df['date'].dt.hour

final_merged_df = pd.get_dummies(data = final_merged_df, columns=['place','month'])
final_merged_df.to_csv(TEST_MERGE+'final_merged.csv',index=False)

In [ ]:
place_lst = [
    '공주',
    '노은동',
    '논산',
    '대천2동',
    '독곶리',
    '동문동',
    '모종동',
    '문창동',
    '성성동',
    '신방동',
    '신흥동',
    '아름동',
    '예산군',
    '읍내동',
    '이원면',
    '정림동',
    '홍성읍'
    ]

In [ ]:
final_df = pd.read_csv(TEST_MERGE+'final_merged.csv')
final_df.drop(['date'], axis=1, inplace=True) # ,'PM2.5'

In [ ]:
df_all = final_df.copy()
df_all.shape

In [ ]:
df_all['month_12'] = 0

In [ ]:
df_all = df_all[['PM2.5', 'temp', 'wind_direction', 'wind_speed', 'rain', 'humid',
       'place_공주', 'place_노은동', 'place_논산', 'place_대천2동', 'place_독곶리',
       'place_동문동', 'place_모종동', 'place_문창동', 'place_성성동', 'place_신방동',
       'place_신흥동', 'place_아름동', 'place_예산군', 'place_읍내동', 'place_이원면',
       'place_정림동', 'place_홍성읍', 'month_1', 'month_2', 'month_3', 'month_4',
       'month_5', 'month_6', 'month_7', 'month_8', 'month_9', 'month_10',
       'month_11', 'month_12','day_1', 'day_2', 'day_3', 'day_4', 'day_5', 'day_6',
       'day_7', 'day_8', 'day_9', 'day_10', 'day_11', 'day_12', 'day_13',
       'day_14', 'day_15', 'day_16', 'day_17', 'day_18', 'day_19', 'day_20',
       'day_21', 'day_22', 'day_23', 'day_24', 'day_25', 'day_26', 'day_27',
       'day_28', 'day_29', 'day_30', 'day_31', 'hour_0', 'hour_1', 'hour_2',
       'hour_3', 'hour_4', 'hour_5', 'hour_6', 'hour_7', 'hour_8', 'hour_9',
       'hour_10', 'hour_11', 'hour_12', 'hour_13', 'hour_14', 'hour_15',
       'hour_16', 'hour_17', 'hour_18', 'hour_19', 'hour_20', 'hour_21',
       'hour_22', 'hour_23']] 

In [ ]:
for place in place_lst:
    globals()['df_{}'.format(place)] = df_all[df_all['place_{}'.format(place)]==1]

In [ ]:
timestep_input = 48
timestep_output = 72
timestep = timestep_input+timestep_output

In [ ]:
Encoder_X_lst = []
encoder_x = []

for place in place_lst:

    df = globals()['df_{}'.format(place)]

    for i in range( len(df)//(timestep_input+timestep_output)): 
        encoder_x.extend([df.iloc[i*timestep : i*timestep + timestep_input, :].values])

Encoder_X = np.array(encoder_x)

In [ ]:
Decoder_X_lst = []
decoder_x = []

for place in place_lst:

    df = globals()['df_{}'.format(place)]

    for i in range( len(df)//(timestep_input+timestep_output) ):
        decoder_x.extend([df.iloc[i*timestep + timestep_input : i*timestep + timestep_input + timestep_output, 6:].values])
        
Decoder_X = np.array(decoder_x).astype('float')

In [ ]:
Decoder_Y_lst = []
decoder_y = []

for place in place_lst:

    df = globals()['df_{}'.format(place)]

    for i in range( len(df)//(timestep_input+timestep_output) ):
        decoder_y.extend([df.iloc[i*timestep + timestep_input : i*timestep + timestep_input + timestep_output, 0].values.T])
        
Decoder_Y = np.array(decoder_y)

In [ ]:
import joblib

LINEAR = 'linear_'
POLYNOMIAL = 'polynomial_'

joblib.dump(Encoder_X,'C:/Users/js/Desktop/AISPRAK/사회문제 해결형/'+LINEAR+'test_encoder_x.pkl')
joblib.dump(Decoder_Y,'C:/Users/js/Desktop/AISPRAK/사회문제 해결형/'+LINEAR+'test_y.pkl')
joblib.dump(Decoder_X,'C:/Users/js/Desktop/AISPRAK/사회문제 해결형/'+LINEAR+'test_decoder_x.pkl')